## Some numbers for thesis

Notebook was generated to calculate some final figures need for the thesis

In [1]:
from pyspark import SparkContext
from os import getcwd
from json import load, loads, dump
from datetime import datetime
import pandas as pd
from glob import glob
from os.path import basename, exists, isfile, isdir, sep



sc = SparkContext(master = 'local[*]')


Loads all required data from the directory which contains all CORE entries with full texts

In [2]:
path = "/home/jovyan/mnt/ceph/storage/data-in-progress/data-research/text-reuse/phoenix/core/core_fulltext/2020_07_10_02_47_49/batches"
if isdir(path):
    filepaths = sorted(glob(path + sep + "*"))

In [3]:
def return_keys(line):
    j = loads(line)
    cid = j['coreId'] if 'coreId' in j else None
    doi = j['doi'] if 'doi' in j else None
    top = j['topics'] if 'topics' in j else None
    lan = j['language'] if 'language' in j else None
    return(cid,doi,top,lan)
    

print(datetime.now())
all_data = []
for p in filepaths:
    print(basename(p))
    rdd = sc.textFile(f"file://{p}")
    data = rdd.map(lambda line: return_keys(line))
    all_data = all_data + data.collect()
    print(datetime.now())

2021-06-25 08:11:08.135356
001
2021-06-25 08:11:56.284934
002
2021-06-25 08:12:35.488326
003
2021-06-25 08:13:47.883469
004
2021-06-25 08:14:49.190157
005
2021-06-25 08:15:26.618577
006
2021-06-25 08:16:06.009011
007
2021-06-25 08:16:30.266930
008
2021-06-25 08:16:42.293781
009
2021-06-25 08:16:53.392192
010
2021-06-25 08:17:32.444792
011
2021-06-25 08:18:47.030197
012
2021-06-25 08:19:14.856891
013
2021-06-25 08:19:58.284725
014
2021-06-25 08:20:57.900095
015
2021-06-25 08:21:23.091743
016
2021-06-25 08:21:45.386209
017
2021-06-25 08:22:55.981110
018
2021-06-25 08:23:44.419730
019
2021-06-25 08:24:38.669169
020
2021-06-25 08:25:12.117669
021
2021-06-25 08:25:50.015862
022
2021-06-25 08:26:24.325161
023
2021-06-25 08:27:45.431290
024
2021-06-25 08:28:56.860284
025
2021-06-25 08:29:52.408174
026
2021-06-25 08:30:29.667608
027
2021-06-25 08:31:26.823297
028
2021-06-25 08:32:25.525694
029
2021-06-25 08:33:18.479408
030
2021-06-25 08:34:09.738835
031
2021-06-25 08:34:34.583607
032
2021-06-

Data is stored in a df. Following numbers are calculated:  

number of entries with DOI  
the number of those DOIs which are unique  
number of entries with topics  
number of entries with language tag  
the total number of unique topics

In [4]:
df = pd.DataFrame(all_data,columns=['id','doi','topics','language'])

In [5]:
len(df['doi'].dropna())

2846221

In [10]:
len(df['doi'].unique())

2474668

In [6]:
sum([len(x) > 0 for x in df['topics']])

6237714

In [7]:
len(df['language'].dropna())

2190342

In [8]:
len(df['topics'].explode().unique())

4591608

Subsequently, we check how many of the fulltext entries are included in the official CORE-MAG mapping

In [9]:
matchtable = pd.read_csv('results/mag_match/2019-04-core-mag.csv.gz', compression='gzip',
                   error_bad_lines=False)
matchtable['coreid'] = matchtable['coreid'].astype('str')
matchtable['magid'] = matchtable['magid'].astype('str')
ids = set(list(df['id']))
s = set(matchtable.coreid)
l = [True if x in s else False for x in ids]
sum(l)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


724497